In [7]:
## Load Data -> Data downloaded from https://data.world/fivethirtyeight/russian-troll-tweets

In [8]:
import pandas as pd
import io
import requests
import re

In [9]:
df1 = pd.read_csv('https://query.data.world/s/vra447dgdpohq4qkuxwijpqoeargjc')
df2 = pd.read_csv('https://query.data.world/s/hyrk2djnl3o4v5yqujcv4rpcihoipk')
df3 = pd.read_csv('https://query.data.world/s/7qv3bcj7nsedi7zlhcelsh5elrqr3v')
df4 = pd.read_csv('https://query.data.world/s/n3cv4borqqb7cwmzjvxqu76xykncar')
df5 = pd.read_csv('https://query.data.world/s/v7uungne76gxhzfthw5b5gqlycl3fx')
df6 = pd.read_csv('https://query.data.world/s/7vql53vrwph5g2hrkb6hdpdyfshnsb')
df7 = pd.read_csv('https://query.data.world/s/mpik75woezhczxttcilmy5jz2gppx6')
df8 = pd.read_csv('https://query.data.world/s/526rhgk3f4nz4d2agfhobhgfasffhm')
df9 = pd.read_csv('https://query.data.world/s/qtfabexnby575a2dsnwya22v3ba36z')


df = pd.DataFrame(df1)
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df = df.append(df5)
df = df.append(df6)
df = df.append(df7)
df = df.append(df8)
df = df.append(df9)

KeyboardInterrupt: 

In [6]:
## Drop columns not needed and count number of tweets in russian just for fun 

df.drop(columns=['external_author_id', 'harvested_date', 'post_type', 'new_june_2018'])

in_russian = 0
is_local = []
for i, row in df.iterrows():
    if row['language'] == 'Russian':
        in_russian+=1
    if (not row['account_type'] == 'left') and (not row['account_type'] == 'right'):
        is_local.append(i)
df.drop(is_local, axis=0, inplace=True)
df.drop(columns=['language'], inplace=True)

print(in_russian)

KeyError: "['language'] not found in axis"

In [ ]:
# Use regex to figure out if the tweet mentions a few relevant political figures 

trump_regex = re.compile(r'(Donald|Trump)')
hill_regex = re.compile(r'(Hillary|Clinton)')
bernie_regex = re.compile(r'(Bernie)')
obama_regex = re.compile(r'(Obama)')
cruz_regex = re.compile(r'(Ted Cruz|Cruz)')
beto_regex = re.compile(r'(Beto|O\'Rourke)')


for i, row in df.head(1000).iterrows():
    df.at[i, 'mention'] = ''
    search_string = str(row['content'])
    if re.search(trump_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Trump '
    if re.search(hill_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Hillary '
    if re.search(bernie_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Bernie '
    if re.search(obama_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Obama '
    if re.search(cruz_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Cruz '
    if re.search(beto_regex, search_string):
        df.at[i, 'mention'] = df.at[i, 'mention'] + 'Beto '

for i, row in df.head(1000).iterrows():
    if df.at[i,'mention'] is '':
        df.at[i,'mention'] = 'NaN'
        
df['mention']